#  Differentiable Model for Muon Transport and Two-Phase Flow in Porous Media with applications to Subsurface Pollution Monitoring

#### Roland Grinis  -  Researcher at MIPT Nuclear Physics Methods lab  -  CTO at GrinisRIT (grinisrit.com)
#### Vladimir Palmin - Researcher at MIPT Nuclear Physics Methods lab 
#### Danila Riazanov - Student at MIPT,  JetBrains Research trainee

## 1. Introduction

Various industrial processes in geological formations, such as carbon dioxide capture sequestration, carbon-enhanced oil recovery, hydraulic fracturing, well disposal etc. could present a risk of groundwater contamination. Non-aqueous phase liquids including chlorinated hydrocarbons, oil, supercritical $\text{CO}_2$ etc. have low solubility in brine. Their migration due to external forces must be thoroughly monitored in order to avoid long-time pollution of freshwater acquifer in the subsurface. See for example:
* J. Rutqvist, *The geomechanics of CO2 storage in deep sedimentary formations.* Geotechnical and Geological Engineering, 30(3) (2012)
* J.G. Wang et al., *A simple approach for the estimation of CO2 penetration depth into a caprock layer.* J. Rock Mech. Geotechn.Eng., 8 (1) (2016)

and references therein specifically for geological carbon storage (GCS).

The use of atmospheric muons to monitor underground fluid saturation levels has been studied before, e.g.:
* F. Hivert et al., *Muography sensitivity to hydrogeological rock density perturbation: roles of the absorption and scattering on the muon flux measurement reliability.* Near Surf. Geophys., 15(2):121 (2017)
* J. Klinger et al., *Simulation of muon radiography for monitoring CO2 stored in a geological reservoir.* Int. J. Greenhouse Gas Control, 42:644 (2015)

However, the low-contrast and possibly noisy muon flux measurements require accurate and realistic modelling of the main physical processes for the inverse problem behind monitoring. Moreover, first order sensitivity information for the parameters of interest is needed to improve the analysis, see:
* C. J. Benton et al. *Optimizing geophysical muon radiography using information theory.* Geophysical Journal International 220.2 (2020)

We address those issues in present work by incorporating a differentiable programming paradigm into the implementation of the detailed physics simulations in our set-up. 

The exposition is organised in the following manner.

In section 2, we describe a model for the two-phase flow with capillary barrier effect in heterogeneuous porous media for which we rely on the mixed-hybrid ﬁnite element method (MHFEM) from:
* R. Fučík et al., *Mixed-hybrid finite element method for modelling two-phase flow in porous media.* Journal of Math-for-Industry 3, no. 2 (2011)
* R. Fučík et al. *Multidimensional mixed–hybrid finite element method for compositional two-phase flow in heterogeneous porous media and its parallel implementation on GPU.* Computer Physics Communications 238 (2019)

We provide there details of our adjoint sensitivity algorithm for MHFEM.

In section 3, we discuss muon transport building upon the Backward Monte-Carlo (BMC) scheme from:
* V. Niess et al., *Backward Monte-Carlo applied to muon transport*, Comput. Phys. Comm.   (2018)

We re-use the spatial discretisation from MHFEM and perform sensitivity computations with respect to the media density and saturation levels following:
* R. Grinis, *Differentiable programming for particle physics simulations.* preprint arXiv:2108.10245 to appear in JETP (2021)

Finally in section 4, we put everything together to design a system for detecting  $\text{CO}_2$ leakage through the caprock layer in GCS sites. 


## 2. Adjoint sensitivity model for two-phase flow

In this section, we consider a system of PDEs in general form relevant to modelling multi-physics multi-phase flow in heterogeneous media with barrier effects. Exact expressions for the coefficients depend on the physics and the modelling set-up. We provide a concrete example in section 4. 

### 2.1 Evolution equations

Let $\boldsymbol{Z}(t,x) \in \mathbb{R}^{n}$ denote the primary unknown variables over $ (t,x) \in \left[0,T\right] \times \varOmega \subset \mathbb{R}^{1+d}$. We pose $\varOmega$ a polygonal $d$-dimensional domain (typically $d=3$) and $T$ finite simulation time.

Introduce the velocity:
$$
\boldsymbol{v}_i = - \sum_{j=1}^{n} \boldsymbol{D}_{ij} \nabla Z_j +  \boldsymbol{w}_i
$$
for $\boldsymbol{D}(t,x,Z) \in \mathbb{R}^{n\times n} \otimes \mathbb{R}^{d}$ the diffusion tensor and $\boldsymbol{w}(t,x,Z) \in \mathbb{R}^{n} \otimes \mathbb{R}^{d}$ the external conservative force.

By convention we denote tensors in bold as opposed to their scalar components, e.g. $\boldsymbol{Z} = \left( Z_i \right)_{i=1}^{n}$, but $\boldsymbol{D} = \left( \boldsymbol{D}_{ij} \right)_{i,j=1}^{n}$.

The conservative flux is given by:
$$
\boldsymbol{q}_i = m_i\boldsymbol{v}_i 
$$
where $\boldsymbol{m}(t,x,Z) \in \mathbb{R}^{n}$ stands for the mobility coefficients.  

The main system governing the evolution is given by:
$$
\sum_{j=1}^{n}\left( N_{ij} \partial_t + \boldsymbol{u}_{ij} \cdot \nabla + r_{ij} \right) Z_j + \nabla \cdot \left( \boldsymbol{q}_i + \sum_{j=1}^{n} Z_j \boldsymbol{a}_{ij} \right) = f_i 
$$

where we the coefficients are:
* $\boldsymbol{N}(t,x,Z) \in \mathbb{R}^{n\times n}$ the damping matrix, 
* $\boldsymbol{r}(t,x,Z) \in \mathbb{R}^{n\times n}$ the reaction matrix, 
* $\boldsymbol{u}(t,x,Z) \in \mathbb{R}^{n\times n} \otimes \mathbb{R}^{d}$ 
the convection tensor for the non-conservative form,
* $\boldsymbol{a}(t,x,Z) \in \mathbb{R}^{n\times n} \otimes \mathbb{R}^{d}$ 
the convection tensor for the conservative form,
* $\boldsymbol{f}(t,x,Z) \in \mathbb{R}^n$ the source/sink term

Moreover we have:
* $\boldsymbol{Z}(0,x) = \boldsymbol{Z}^{\mathcal{I}}(x)$, $\forall x \in \varOmega$ (initial conditions)
* $Z_i(t,x) = Z_i^{\mathcal{D}}(t,x)$, $\forall x \in \varGamma_{Z_i} \subset \partial\varOmega$ (Dirichlet boundary conditions)
* $\boldsymbol{q}_i \cdot \boldsymbol{n} = q_i^{\mathcal{N}}$, $\forall x \in \varGamma_{\boldsymbol{q}_i} \subset \partial\varOmega$ with $\boldsymbol{n}$ outward pointing normal to $\partial\varOmega$ (Neumann boundary conditions)

The function $\boldsymbol{Z}$ is required to be continuously differentiable in time only and satisfies the evolution equations in the weak sense.

### 2.1 Mixed-hybrid finite element method

Let $\mathcal{K}_h$ be a conformal mesh for $\varOmega$ with $\mu_d(K) = \mathcal{O}(h^d)$, $\forall K \in \mathcal{K}_h$, where $\mu_d$ stands for the Lebesgue measure on $\mathbb{R}^{d}$. Denote by $\mathcal{V}_h$ the set of vertices and by $\mathcal{E}_h$ the set of ($d-1$)-dimensional sides composing $\mathcal{K}_h$. We split the latter into interior
$\mathcal{E}_h^{int}$ and exterior $\mathcal{E}_h^{ext}$ sides. For a given element $K \in \mathcal{K}_h$ we write $\mathcal{E}_K$ for the set of sides forming $K$. 



## 3. Sensitivity analysis for muon transport 

## 4. Leakage monitoring in GCS